In [173]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from tqdm.auto import tqdm
from datetime import datetime

from bs4 import BeautifulSoup
import requests
from lxml import etree
from time import sleep

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [3]:
nasdaq_df = pd.read_csv('./data/nasdaq_list.csv')

In [4]:
def scrape_finbiz(symbol: str):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        resp = requests.get('https://finviz.com/quote.ashx?t=' + symbol, headers=headers, timeout=15)
        soup = BeautifulSoup(resp.text)
    except:
        print(symbol, "request failed.")
        return None

    try:
        snapshot_table2 = soup.find('table', attrs={'class': 'snapshot-table2'})
        tables = pd.read_html(str(snapshot_table2))

        df = tables[0]

        market_cap = df.iloc[1, 1]
        income = df.iloc[2, 1]
        sales = df.iloc[3, 1]
        gross_margin = df.iloc[7, 7]
        operation_margin = df.iloc[8, 7]
        profit_margin = df.iloc[9, 7]
        employees = df.iloc[8, 1]

        series = [market_cap, income, sales, gross_margin, operation_margin, profit_margin, employees]
        return series
    except:
        print(symbol, "parsing failed.")
        return None


In [5]:
def thread_scrape(start: int, end: int):
    result = []
    for i in tqdm(range(start, end)):
        name = nasdaq_df.loc[i, 'Name']
        symbol = nasdaq_df.loc[i, 'Symbol']
        industry = nasdaq_df.loc[i, 'Industry']
        
        data = [name, symbol, industry]
        scraped = scrape_finbiz(symbol)
        if scraped is not None:
            data.extend(scraped)
            result.append(data)
    
    return result

In [219]:
work_list = [(0, 10)]
# for i in range(0, len(nasdaq_df), 500):
#     work_list.append((i, min(i + 500, len(nasdaq_df))))
# print(work_list)

with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(thread_scrape, work[0], work[1]) for work in work_list]

    df = []
    for future in as_completed(futures):
        df.extend(future.result())

df = pd.DataFrame(
    df, 
    columns=['Name', 'Symbol', 'Industry', 'MarketCap', 'Income', 'Sales', 'GrossMargin', 'OperatingMargin', 'ProfitMargin', 'Employees']
)
df.to_csv('./data/nasdaq_marketcap_full.csv')
df

  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: 10 columns passed, passed data had 9 columns

In [13]:
import re

def _conv_to_float(s):
    if s == '-':
        return None

    if s[-1] == '%':
        s = s.replace('%', '')
    if s[-1] in list('BMK'):
        powers = {'B': 10 ** 9, 'M': 10 ** 6, 'K': 10 ** 3, '': 1}
        m = re.search("([0-9\.]+)(M|B|K|)", s)
        if m:
            val, mag = m.group(1), m.group(2)
            return float(val) * powers[mag]
    try:
        result = float(s)
    except:
        result = None
    return result

In [33]:
df = pd.read_csv('./data/nasdaq_marketcap_full.csv')
df = df.drop(df.columns[[0]], axis=1)

conv_list =['MarketCap', 'Income', 'Sales', 'GrossMargin', 'OperatingMargin', 'ProfitMargin']
for col in conv_list:
    df[col] = df[col].apply(_conv_to_float)

nasdaq_df_proc = df.dropna(subset=['MarketCap'])

nasdaq_df_proc

,Name,Symbol,Industry,MarketCap,Income,Sales,GrossMargin,OperatingMargin,ProfitMargin
0,Investors Title Co,ITIC,보험,289840000.0,59400000.0,320600000.0,NaN,21.7,18.5
1,Capital Product Partners Units,CPLP,오일 및 가스 장비 및 서비스,281720000.0,0.0,219900000.0,94.5,62.4,0.0
2,Northwest Pipe Co,NWPX,메탈&마이닝,288260000.0,12900000.0,370300000.0,13.6,5.0,3.5
3,Noble Rock Acquisition Corp Class A,NRAC,기타 금융업,296070000.0,11100000.0,NaN,NaN,NaN,NaN
5,bluebird bio Inc,BLUE,생명과학 및 메디컬 리서치,349010000.0,563300000.0,4700000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3626,Fednat Holding Co,FNHC,보험,5550000.0,127600000.0,235000000.0,NaN,-48.7,-54.3
3627,China SXT Pharmaceuticals Inc,SXTC,제약,5410000.0,7200000.0,1900000.0,17.6,NaN,NaN
3628,Baudax Bio Inc,BXRX,생명과학 및 메디컬 리서치,5330000.0,18100000.0,1300000.0,-74.2,NaN,NaN
3629,Autoweb Inc,AUTO,미디어 및 출판,5520000.0,10300000.0,72800000.0,25.3,-12.7,-14.1


In [162]:
top10_df = pd.DataFrame(nasdaq_df_proc, columns=['Industry', 'MarketCap']).groupby('Industry').sum()
top10_df = top10_df.sort_values(by=['MarketCap'], ascending=False)[:10]

df = nasdaq_df_proc.loc[nasdaq_df_proc['Industry'].isin(top10_df.index)]
df = df.sort_values(by=['MarketCap'], ascending=False)

dummy_data = []
for industry in reversed(top10_df.index):
    dummy_data.insert(0, ['dummy', 'DUMMY', industry, 0, 0, 0, 0, 0, 0])
df = pd.concat([pd.DataFrame(dummy_data, columns=df.columns), df])

fig = px.bar(
    df,
    x='Industry',
    y='MarketCap',
    color='Name',
    barmode='stack',
    title='NASDAQ 산업별 시가총액 총합'
)
fig.update_layout(
    showlegend=False,
)
fig.show()

In [99]:
count_dict = nasdaq_df_proc['Industry'].value_counts().to_dict()

top5_df = pd.DataFrame(nasdaq_df_proc, columns=['Industry', 'MarketCap']).groupby('Industry').sum().sort_values(by=['MarketCap'], ascending=False)[:5]
top5_df['Industry'] = top5_df.index
top5_df['Count'] = top5_df['Industry'].map(lambda x: count_dict[x])
# print(top5_df)
print(top5_df.describe())

corr = top5_df.corr(method='pearson')
# corr = top5_df.corr(method='kendall')
corr

          MarketCap       Count
count  5.000000e+00    5.000000
mean   2.861433e+12  115.800000
std    2.727506e+12  160.348683
min    8.506335e+11   20.000000
25%    1.496068e+12   31.000000
50%    2.000252e+12   42.000000
75%    2.322057e+12   87.000000
max    7.638153e+12  399.000000


,MarketCap,Count
MarketCap,1.000000,0.973191
Count,0.973191,1.000000


In [204]:
soft_df = pd.read_csv('./data/nasdaq_marketcap_full.csv')
soft_df = soft_df.drop(soft_df.columns[[0]], axis=1)

conv_list =['MarketCap', 'Income', 'Sales', 'GrossMargin', 'OperatingMargin', 'ProfitMargin']
soft_df = soft_df[soft_df['Industry'] == '소프트웨어 및 IT서비스']
for col in conv_list:
    soft_df[col] = soft_df[col].apply(_conv_to_float)
soft_df['NewsTimeDense'] = 0
soft_df = soft_df.reset_index(drop=True)
soft_df

,Name,Symbol,Industry,MarketCap,Income,Sales,GrossMargin,OperatingMargin,ProfitMargin,NewsTimeDense
0,Outbrain Inc,OB,소프트웨어 및 IT서비스,308740000.0,4900000.0,1.040000e+09,23.1,NaN,NaN,0
1,Cognyte Software Ltd,CGNT,소프트웨어 및 IT서비스,301960000.0,40900000.0,4.458000e+08,69.8,-3.7,-9.2,0
2,Digimarc Corp,DMRC,소프트웨어 및 IT서비스,289510000.0,44300000.0,2.720000e+07,60.2,NaN,NaN,0
3,EverQuote Inc Class A,EVER,소프트웨어 및 IT서비스,282550000.0,21300000.0,4.254000e+08,94.4,-5.6,-5.0,0
4,PFSweb Inc,PFSW,소프트웨어 및 IT서비스,265440000.0,19700000.0,2.831000e+08,16.7,-6.2,50.2,0
...,...,...,...,...,...,...,...,...,...,...
395,MMTEC Inc,MTC,소프트웨어 및 IT서비스,9300000.0,7100000.0,7.000000e+05,78.4,NaN,NaN,0
396,Versus Systems Inc,VS,소프트웨어 및 IT서비스,8660000.0,14300000.0,8.000000e+05,NaN,NaN,NaN,0
397,Grom Social Enterprises Inc,GROM,소프트웨어 및 IT서비스,7790000.0,11900000.0,5.700000e+06,32.4,NaN,NaN,0
398,Liquid Media Group Ltd,YVR,소프트웨어 및 IT서비스,7440000.0,12100000.0,1.300000e+06,12.2,NaN,NaN,0


In [198]:
headers = {'User-Agent': 'Mozilla/5.0'}

for i in tqdm(range(len(soft_df))):
    symbol = soft_df.loc[i, 'Symbol']

    resp = requests.get('https://finviz.com/quote.ashx?t=' + symbol, headers=headers, timeout=15)
    soup = BeautifulSoup(resp.text)
    snapshot_table2 = soup.find('table', attrs={'class': 'fullview-news-outer'})
    tables = pd.read_html(str(snapshot_table2))
    df = tables[0]

    full_dates = []
    for j in range(len(df)):
        t = df.iloc[j, 0]
        if len(t.split(' ')) == 2:
            full_dates.append(t)

    start = datetime.strptime(full_dates[0], '%b-%d-%y %I:%M%p')
    if len(full_dates[-1].split(' ')) == 2:
        end = datetime.strptime(full_dates[-1], '%b-%d-%y %I:%M%p')
    else:
        end = datetime.strptime(full_dates[-1].split(' ')[0] + ' ' + df.iloc[-1, 0], '%b-%d-%y %I:%M%p')
    time_diff = start - end
    # print(symbol, time_diff.seconds, len(df))
    soft_df.loc[i, 'NewsTimeDense'] = time_diff.seconds / len(df)
    # soft_df.loc[i, 'NewsTimeDenseInv'] = len(df) / time_diff.seconds

soft_df

  0%|          | 0/400 [00:00<?, ?it/s]

,Name,Symbol,Industry,MarketCap,Income,Sales,GrossMargin,OperatingMargin,ProfitMargin,NewsTimeDense,NewsTimeDenseInv
0,Outbrain Inc,OB,소프트웨어 및 IT서비스,308740000.0,4900000.0,1.040000e+09,23.1,NaN,NaN,739.591837,0.001352
1,Cognyte Software Ltd,CGNT,소프트웨어 및 IT서비스,301960000.0,40900000.0,4.458000e+08,69.8,-3.7,-9.2,2454.545455,0.000407
2,Digimarc Corp,DMRC,소프트웨어 및 IT서비스,289510000.0,44300000.0,2.720000e+07,60.2,NaN,NaN,36.000000,0.027778
3,EverQuote Inc Class A,EVER,소프트웨어 및 IT서비스,282550000.0,21300000.0,4.254000e+08,94.4,-5.6,-5.0,114.600000,0.008726
4,PFSweb Inc,PFSW,소프트웨어 및 IT서비스,265440000.0,19700000.0,2.831000e+08,16.7,-6.2,50.2,591.000000,0.001692
...,...,...,...,...,...,...,...,...,...,...,...
395,MMTEC Inc,MTC,소프트웨어 및 IT서비스,9300000.0,7100000.0,7.000000e+05,78.4,NaN,NaN,1336.800000,NaN
396,Versus Systems Inc,VS,소프트웨어 및 IT서비스,8660000.0,14300000.0,8.000000e+05,NaN,NaN,NaN,487.500000,NaN
397,Grom Social Enterprises Inc,GROM,소프트웨어 및 IT서비스,7790000.0,11900000.0,5.700000e+06,32.4,NaN,NaN,70.666667,NaN
398,Liquid Media Group Ltd,YVR,소프트웨어 및 IT서비스,7440000.0,12100000.0,1.300000e+06,12.2,NaN,NaN,18.000000,NaN


In [220]:
df = pd.read_csv('./data/nasdaq_timedense.csv')
df.describe()

,Unnamed: 0,MarketCap,Income,Sales,GrossMargin,OperatingMargin,ProfitMargin,NewsTimeDense
count,400.000000,3.990000e+02,3.730000e+02,3.940000e+02,353.000000,309.000000,303.000000,400.000000
mean,199.500000,1.914324e+10,6.976268e+08,3.374240e+09,55.062606,-9.543366,-10.566337,813.854517
std,115.614301,1.473239e+11,5.713021e+09,2.242838e+10,24.082465,27.848631,27.332557,1334.554790
min,0.000000,5.210000e+06,1.000000e+05,1.000000e+04,-23.300000,-99.300000,-99.400000,0.000000
25%,99.750000,6.930000e+07,1.270000e+07,5.445000e+07,37.600000,-25.400000,-25.050000,230.400000
50%,199.500000,5.635900e+08,4.570000e+07,2.834500e+08,60.100000,-4.500000,-4.500000,590.100000
75%,299.250000,3.430000e+09,1.268000e+08,9.663000e+08,74.100000,8.300000,7.150000,837.600000
max,399.000000,1.945040e+12,7.454000e+10,2.703300e+11,99.800000,65.100000,58.600000,17100.000000


In [218]:
import plotly.figure_factory
def draw_heatemap(_df: pd.DataFrame, title: str):
    df_corr = _df.dropna().corr()

    x = list(df_corr.columns)
    y = list(df_corr.index)
    z = np.array(df_corr)

    fig = plotly.figure_factory.create_annotated_heatmap(
        z,
        x=x,
        y=y,
        annotation_text=np.around(z, decimals=2),
        hoverinfo='z',
        colorscale='Viridis',
    )
    fig.update_layout(width=800, height=800, title=title)
    fig.show()

soft_df = pd.read_csv('./data/nasdaq_timedense.csv')
soft_df = soft_df.drop(soft_df.columns[[0]], axis=1)
draw_heatemap(soft_df, '재무제표 상관관계')
draw_heatemap(soft_df.sort_values(by=['MarketCap'], ascending=False)[:4], '재무제표 상관관계 - 상위4개')
draw_heatemap(soft_df.sort_values(by=['MarketCap'], ascending=False)[4:], '재무제표 상관관계 - 상위4개 제외')
draw_heatemap(soft_df.sort_values(by=['MarketCap'], ascending=False)[len(soft_df)-100:], '재무제표 상관관계 - 하위100')